<a href="https://colab.research.google.com/github/ashish-3916/machine-learning-/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np  

In [2]:
def fit (x_train , y_train):
  result = {}
  class_values = set(y_train)  # final attribute values
  for current_class in class_values:
    result[current_class] = {}
    result["total_data"] = len(y_train)

    # take only those rows which has this label as answer
    current_class_rows = (y_train == current_class )
    x_train_current = x_train[current_class_rows]
    y_train_current = y_train[current_class_rows]

    # work for every feature 
    num_features = x_train.shape[1]
    result[current_class]["total_count"] = len(y_train_current)

    for j in range(1 , num_features +1 ):
      result[current_class][j] = {}
      # all possible values these features can take 
      all_possible_values = set(x_train[: , j-1])
      for current_value in all_possible_values:
        result[current_class][j][current_value] = (x_train_current[: , j-1]==current_value).sum()

    return result 

In [3]:

# def probabilty(dict , x , current_class):
#   output = 1;
#   # p(X = x | y = current_class ) * p( y = current_class)
#   #             feature prob      *   class_prob
#   class_prob = dict[current_class]["total_count"] / dict["total_data"]
#   feature_prob = 1;
#   num_features = len(dict[current_class].keys()) - 1
#   for j in range (1 , num_features+1):
#     xj =  x[j-1]
#     count_current_class_with_value_xj =  dict[current_class][j][xj] + 1 # laplace correction
#     count_current_class = dict[current_class]["total_count"] + len(dict[current_class][j].keys()) #laplace correction
#     current_p =  count_current_class_with_value_xj / count_current_class
#     feature_prob*= current_p
  
#   output = feature_prob * class_prob

#   return output




In [4]:
#changing to log probablities bcz in multiplication they are already so small 

def probabilty(dict , x , current_class):
  # output = 1;
  # p(X = x | y = current_class ) * p( y = current_class)
  #             feature prob      *   class_prob
  class_prob = np.log(dict[current_class]["total_count"]) - np.log(dict["total_data"])
  feature_prob = 1;
  num_features = len(dict[current_class].keys()) - 1
  for j in range (1 , num_features+1):
    xj =  x[j-1]
    count_current_class_with_value_xj =  dict[current_class][j][xj] + 1 # laplace correction
    count_current_class = dict[current_class]["total_count"] + len(dict[current_class][j].keys()) #laplace correction
    current_p =  np.log(count_current_class_with_value_xj) - np.log(count_current_class) 
    class_prob += current_p
  
  # output = feature_prob * class_prob

  return class_prob


In [5]:

def find(dict , x) :
  classes =  dict.keys()
  best_prob = -1000
  best_class = -1

  for current_class in classes :
    if (current_class == "total_data") : 
      continue
    p_current_class = probabilty(dict , x , current_class)
    if( p_current_class  > best_prob) :
      best_prob = p_current_class
      best_class = current_class
    
  return best_class 

In [6]:
def predict(dict , x_test):
  y_pred = []
  for x in x_test : 
    x_class = find(dict , x)
    y_pred.append(x_class)
  return y_pred 

okay now lets test our code

In [7]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0,len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [8]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [9]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [10]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [11]:
dictionary = fit(X_train,Y_train)

Y_pred = predict(dictionary,X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.34      1.00      0.51        13
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         9

    accuracy                           0.34        38
   macro avg       0.11      0.33      0.17        38
weighted avg       0.12      0.34      0.17        38

[[13  0  0]
 [16  0  0]
 [ 9  0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
